In [ ]:
status_response = {'reference': 'ref-15149-57642', 'event': 'verification.accepted', 'country': 'AU', 'proofs': {'document': {'proof': 'https://ns.shuftipro.com/api/pea/efdc3b3f8cecc05fd782b5378e687c73d659e271', 'additional_proof': 'https://ns.shuftipro.com/api/pea/c4b99d49f7e6b8bc05329f0876da1404861f2c04'}, 'access_token': 'test', 'verification_report': 'https://ns.shuftipro.com/api/pea/ac69902212ae4beb90b6e2be574d2a3e21401852'}, 'email': 'johndoe@example.com', 'verification_data': {'document': {'name': {'first_name': 'LIVONE', 'middle_name': None, 'last_name': 'JOHN'}, 'dob': '1986-09-06', 'expiry_date': '2030-11-12', 'issue_date': '2020-11-12', 'document_number': 'A123456', 'gender': 'M', 'place_of_issue': 'United Kingdom', 'selected_type': ['driving_license'], 'supported_types': ['id_card', 'passport', 'driving_license']}}, 'verification_result': {'document': {'document': 1, 'document_visibility': 1, 'document_must_not_be_expired': 1, 'document_proof': 1, 'selected_type': 1, 'name': 1, 'dob': 1, 'gender': 1, 'place_of_issue': 1, 'document_number': 1, 'expiry_date': 1, 'issue_date': 1}}, 'info': {'agent': {'is_desktop': True, 'is_phone': False, 'useragent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 OPR/122.0.0.0', 'device_name': 'Windows NT 10.0', 'browser_name': 'Opera 122.0.0.0', 'platform_name': 'Windows 10'}, 'geolocation': {'host': '2001:8003:cd18:c600:6ce7:974d:c29e:ef89', 'ip': '2001:8003:cd18:c600:6ce7:974d:c29e:ef89', 'rdns': '2001:8003:cd18:c600:6ce7:974d:c29e:ef89', 'asn': '1221', 'isp': 'Telstra Limited', 'country_name': 'Australia', 'country_code': 'AU', 'region_name': 'Victoria', 'region_code': 'VIC', 'city': 'Melbourne', 'postal_code': '8070', 'continent_name': 'Oceania', 'continent_code': 'OC', 'latitude': '-37.813201904297', 'longitude': '144.96299743652', 'metro_code': '', 'timezone': 'Australia/Melbourne', 'ip_type': 'ipv6', 'capital': 'Canberra', 'currency': 'AUD'}}, 'additional_data': {'document': {'proof': {'document_number': 'A123456', 'full_name': 'John Livone', 'dob': '1986-09-06', 'issue_date': '2020-11-12', 'expiry_date': '2030-11-12', 'gender': 'M', 'place_of_issue': 'United Kingdom', 'address': '10 Downing st, Westminster, London SW1A 2AA, UK', 'document_country': 'United Kingdom', 'document_country_code': 'GB', 'country': 'United Kingdom', 'country_native': 'United Kingdom of Great Britain and Northern Ireland', 'country_code': 'GB', 'document_official_name': 'ShuftiPro Demo ID Card version-1', 'signature': '0,0,0,0', 'face': '42,121,258,432'}, 'additional_proof': {'address': 'ABCD Street, Box 12256, SE-102 26 XYZ, City', 'document_number': 'A123456', 'last_name': 'JOHN', 'first_name': 'LIVONE', 'full_name': 'JOHN LIVONE', 'dob': '1986-09-06', 'expiry_date': '2030-11-12', 'document_official_name': 'ShuftiPro Demo ID Card BackSide version-1', 'document_country': 'United States', 'document_country_code': 'US', 'authority': 'United States Department of state', 'country': 'United States', 'country_native': 'United States of America', 'country_code': 'US', 'mrz': 'SP<<A123456<<<<<<<<<<<<<\n860906<<<<301112<<<<<<<<\nJOHN<<<LIVONE<<<<<<<<<<<', 'signature': '124,165,241,229', 'face': '0,0,0,0'}}}}

In [2]:
import requests
def download_proof_image(proof_url: str, access_token: str):
    """Download proof image using access token"""
    payload = {
        "access_token": access_token
    }
    
    response = requests.post(
        proof_url,
        json=payload,
        headers={"Content-Type": "application/json"}
    )
    
    if response.status_code == 200:
        return response.content
    else:
        print(f"Failed to download proof: {response.status_code}")
        return None

In [7]:
proofs = status_response.get('proofs', {})
access_token = proofs.get('access_token')
document_proofs = proofs.get('document', {})
front_url = document_proofs.get('proof')
back_url = document_proofs.get('additional_proof')

front_image = download_proof_image(front_url, access_token)
filename = f"front.jpg"
filepath = f"{filename}"

with open(filepath, 'wb') as f:
    f.write(front_image)

print(f"✅ Downloaded front image: {filename}")

✅ Downloaded front image: front.jpg


In [3]:
import os
import base64
import json
import hashlib
def get_verification_status_with_proofs(reference: str):
    """Call Status API to get proof URLs and access token"""
    url = 'https://api.shuftipro.com/status'
    client_id = os.environ.get("SHUFTI_CLIENTID")
    secret_key = os.environ.get("SHUFTI_SECRET_KEY")
    
    payload = {
        "reference": reference
    }
    
    auth = f'{client_id}:{secret_key}'
    b64Val = base64.b64encode(auth.encode()).decode()
    
    response = requests.post(
        url,
        headers={
            "Authorization": f"Basic {b64Val}",
            "Content-Type": "application/json"
        },
        data=json.dumps(payload)
    )
    
    # Verify signature
    secret_key_hash = hashlib.sha256(secret_key.encode()).hexdigest()
    calculated_signature = hashlib.sha256(
        f"{response.content.decode()}{secret_key_hash}".encode()
    ).hexdigest()
    sp_signature = response.headers.get('Signature', "")
    
    if sp_signature == calculated_signature:
        return response.json()
    else:
        print(f"Invalid signature in status response")
        return None
